# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbc2ca63790>
├── 'a' --> tensor([[ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772]])
└── 'x' --> <FastTreeValue 0x7fbc2cacdee0>
    └── 'c' --> tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                        [ 0.8785, -2.0157, -0.2295,  0.8163],
                        [ 1.1909,  0.5739, -0.6907,  1.9536]])

In [4]:
t.a

tensor([[ 0.1035,  1.2062,  0.8620],
        [ 0.1784,  1.1038, -0.8772]])

In [5]:
%timeit t.a

57.8 ns ± 0.0672 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbc2ca63790>
├── 'a' --> tensor([[ 0.3584,  0.6018,  0.6040],
│                   [-0.2479,  1.7502, -0.2814]])
└── 'x' --> <FastTreeValue 0x7fbc2cacdee0>
    └── 'c' --> tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                        [ 0.8785, -2.0157, -0.2295,  0.8163],
                        [ 1.1909,  0.5739, -0.6907,  1.9536]])

In [7]:
%timeit t.a = new_value

66 ns ± 0.0581 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1035,  1.2062,  0.8620],
               [ 0.1784,  1.1038, -0.8772]]),
    x: Batch(
           c: tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1035,  1.2062,  0.8620],
        [ 0.1784,  1.1038, -0.8772]])

In [11]:
%timeit b.a

59.4 ns ± 0.0196 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2930, -1.8480, -0.4335],
               [ 0.2767, -1.2644,  1.1554]]),
    x: Batch(
           c: tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

839 ns ± 0.0524 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 637 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbb61027dc0>
├── 'a' --> tensor([[[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]],
│           
│                   [[ 0.1035,  1.2062,  0.8620],
│                    [ 0.1784,  1.1038, -0.8772]]])
└── 'x' --> <FastTreeValue 0x7fbb610279a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbb610279d0>
├── 'a' --> tensor([[ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772],
│                   [ 0.1035,  1.2062,  0.8620],
│                   [ 0.1784,  1.1038, -0.8772]])
└── 'x' --> <FastTreeValue 0x7fbc2ca63460>
    └── 'c' --> tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                        [ 0.8785, -2.0157, -0.2295,  0.8163],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 39.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.7 µs ± 52.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2846, -1.4154,  0.0169,  0.2447],
                       [ 0.8785, -2.0157, -0.2295,  0.8163],
                       [ 1.1909,  0.5739, -0.6907,  1.9536]],
              
                      [[-1.2846, -1.4154,  0.0169,  0.2447],
                       [ 0.8785, -2.0157, -0.2295,  0.8163],
                       [ 1.1909,  0.5739, -0.6907,  1.9536]],
              
                      [[-1.2846, -1.4154,  0.0169,  0.2447],
                       [ 0.8785, -2.0157, -0.2295,  0.8163],
                       [ 1.1909,  0.5739, -0.6907,  1.9536]],
              
                      [[-1.2846, -1.4154,  0.0169,  0.2447],
                       [ 0.8785, -2.0157, -0.2295,  0.8163],
                       [ 1.1909,  0.5739, -0.6907,  1.9536]],
              
                      [[-1.2846, -1.4154,  0.0169,  0.2447],
                       [ 0.8785, -2.0157, -0.2295,  0.8163],
                       [ 1.1909,  0.5739, -0.6907,  1.9536]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536],
                      [-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536],
                      [-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536],
                      [-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536],
                      [-1.2846, -1.4154,  0.0169,  0.2447],
                      [ 0.8785, -2.0157, -0.2295,  0.8163],
                      [ 1.1909,  0.5739, -0.6907,  1.9536],
                      [-1.2846, -1.4154,  0.0169,  0.2447],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 334 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
